In [131]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
from google.colab import drive
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
currency_type = 'ETH'

In [ ]:
### Code for pure transactions datasets

data = pd.read_csv('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/'+str(0)+'.csv')
for k in range(1, 12):
  data = data.append(pd.read_csv('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/'+str(k)+'.csv'))
data['label'] = 0
data['label'][data['m'] == True] = 1
times_start = data['T'][:-1].to_numpy()
times_end = data['T'][1:].to_numpy()
inter_event = times_end - times_start
since_start = np.cumsum(inter_event)
transact_dataset = np.concatenate((since_start.reshape(-1, 1), inter_event.reshape(-1, 1), data['label'][1:].to_numpy().reshape(-1, 1)), axis = 1)
np.save('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/transact_dataset_' + currency_type, transact_dataset)

In [ ]:
### Code for LOB transactions datasets
for n in [0, 1]:
  LOB = []
  for k in range(n*6, (n+1)*6):
    data = pd.read_csv('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/'+str(k)+'.csv')
    data = data[['T', 'm', 'p']]

    temp_ask = []

    temp_bid = []

    LOB_new = []
    for i in data.index:

      if data['m'][i] == True and len(temp_bid) < 10:
        temp_bid += [data['p'][i]]
        if len(temp_bid) == 1:
          time_1 = data['T'][i]

      if data['m'][i] == False and len(temp_ask) < 10:
        temp_ask += [data['p'][i]]
        if len(temp_ask) == 1:
          time_2 = data['T'][i]
      if len(temp_bid) == 10 and len(temp_ask) == 10:
        if time_1 < time_2:
          time = time_1
        else:
          time = time_2
        LOB_new += [[time, sorted(temp_bid), sorted(temp_ask, reverse = True)]]
        temp_ask = []
        temp_bid = []
    LOB += LOB_new
    print(len(LOB))
  LOB = np.array(LOB)
  np.save('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/LOB_'+str(n+1), LOB)


In [ ]:
# Datasets are too massive, devide them into two files
LOB_1 = np.load('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/LOB_1.npy', allow_pickle= True)
LOB_2 = np.load('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/LOB_2.npy', allow_pickle= True)

In [ ]:
LOB = np.concatenate((LOB_1, LOB_2), axis = 0)

In [ ]:
### Code for the final form of datasets, with times, inter-event times and types of events
new_array = []
for i in LOB:
  new_array += [[i[0], np.mean([i[1][0], i[2][0]])]]
new_array = np.array(new_array)

T = new_array[:,:1]
p = new_array[:,1:]
inter_event = T[1:] - T[:-1]
since_last_event = np.cumsum(inter_event).reshape((-1, 1))
price_change = p[1:] - p[:-1]

label = []
for i in price_change:
  if i > 0:
    label += [1]
  else:
    label += [0]
label = np.array(label).reshape(-1, 1)
dataset = np.concatenate((inter_event, since_last_event, label), axis = 1)

In [ ]:
np.save('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/dataset_' + currency_type, dataset)

In [134]:
dataset = np.load('/content/drive/MyDrive/Sequential_datasets/' + currency_type + '/dataset_' + currency_type + '.npy', allow_pickle= True)

In [135]:
### Deviding datasets into sequences, 3000 rows in each
n_batches = int(dataset.shape[0]/3000)
batched = dataset[:n_batches*3000].reshape(n_batches, 3000, 3)

### Filtering the anomal data
times = batched[:, :, 0]
Q1 = np.percentile(times, 25)
Q3 = np.percentile(times, 75)
IQR = Q3 - Q1

new_array = np.array([]).reshape(-1, 3000, 3)
for batch in batched:
  if all(Q1 - 10*IQR <= batch[:, 0]) and all(batch[:, 0] <= Q3 + 10*IQR):
      new_array = np.concatenate((new_array, batch.reshape(-1, 3000, 3)), axis = 0)

In [ ]:
dataset

In [ ]:
dataset.shape[0]/3000

300.663

In [ ]:
batched.shape

(300, 3000, 3)

In [ ]:
new_array.shape

(260, 3000, 3)

In [ ]:
new_array[:,:,0].max()

387982.0

In [ ]:
np.save('/content/drive/MyDrive/Sequential_datasets_2/' + currency_type + '/dataset_filtered_' + currency_type, new_array)